In [1]:
import os

In [2]:
%pwd


'd:\\Python\\End-To-End Ml Project\\End-to-end-ML-Project-with-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Python\\End-To-End Ml Project\\End-to-end-ML-Project-with-MLflow'

In [17]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file :Path
    unzip_dir: Path

In [18]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:

    def __init__(self,
                config_filepath = CONFIG_FILE_PATH, # reading config/config.yaml file 
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
         
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root]) # create artifacts folder

    
    # DataIngestionConfig - is custom entity(datatype) created for return purspose(custom return type)
    def get_data_ingestion_config(self)-> DataIngestionConfig: # this fun only reads config file for data ingestion

        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir =config.unzip_dir
        )

        return data_ingestion_config

In [20]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size 

In [21]:
class DataIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info : \n {headers}")
        else:
            logger.info(f"file alreadt exists of size:{get_size(Path(self.config.local_data_file))}")   



    def extract_zip_file(self):
        
        unzip_path =self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [22]:
# read the data using pipeline(setp by step)

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2023-10-22 16:14:18,157: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-22 16:14:18,159: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-22 16:14:18,162: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-22 16:14:18,163: INFO: common: created directory at artifacts]
[2023-10-22 16:14:18,166: INFO: common: created directory at artifacts/data_ingestion]


[2023-10-22 16:14:18,985: INFO: <ipython-input-21-23f0f8f4be85>: artifacts/data_ingestion/data.zip download! with following info : 
 Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 19E2:939A:799B72:7EFF06:65353C49
Accept-Ranges: bytes
Date: Sun, 22 Oct 2023 15:14:18 GMT
Via: 1.1 varnish
X-Served-By: cache-lon4281-LON
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1697987658.950907,VS0,VE236
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 1ff9064c8254778a4addcfbfd446f744a51e0d74
Expires: Sun, 22 Oct 2023 15:19:18 GMT
Source-Age: 0

]
